## Initial exploratory analysis

In [7]:
## importing libraries
import pandas as pd
import numpy as np
import geopy.distance 
from dateutil.relativedelta import relativedelta
from datetime import datetime
import os

## Here we set the directory to look up the folder containing the data
os.chdir("/Users/gabrielmedeiros/Documents/OneDrive/Business analytics/DATA_445_Machine_Learning")


## Reading csv file, index_col = 0 makes the first column of the data to become the index of our pandas data frame
test_data = pd.read_csv('fraudTest.csv', index_col = 0)

train_data = pd.read_csv('fraudTrain.csv', index_col = 0)

## Head of Train data

In [8]:
print(train_data.shape)
train_data.head()

(1296675, 22)


,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0


## Head of Test Data

In [9]:
print(test_data.shape)
test_data.head()

(555719, 22)


,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2020-06-21 12:14:25,2291163933867244,fraud_Kirlin and Sons,personal_care,2.86,Jeff,Elliott,M,351 Darlene Green,Columbia,...,33.9659,-80.9355,333497,Mechanical engineer,1968-03-19,2da90c7d74bd46a0caf3777415b3ebd3,1371816865,33.986391,-81.200714,0
1,2020-06-21 12:14:33,3573030041201292,fraud_Sporer-Keebler,personal_care,29.84,Joanne,Williams,F,3638 Marsh Union,Altonah,...,40.3207,-110.4360,302,"Sales professional, IT",1990-01-17,324cc204407e99f51b0d6ca0055005e7,1371816873,39.450498,-109.960431,0
2,2020-06-21 12:14:53,3598215285024754,"fraud_Swaniawski, Nitzsche and Welch",health_fitness,41.28,Ashley,Lopez,F,9333 Valentine Point,Bellmore,...,40.6729,-73.5365,34496,"Librarian, public",1970-10-21,c81755dbbbea9d5c77f094348a7579be,1371816893,40.495810,-74.196111,0
3,2020-06-21 12:15:15,3591919803438423,fraud_Haley Group,misc_pos,60.05,Brian,Williams,M,32941 Krystal Mill Apt. 552,Titusville,...,28.5697,-80.8191,54767,Set designer,1987-07-25,2159175b9efe66dc301f149d3d5abf8c,1371816915,28.812398,-80.883061,0
4,2020-06-21 12:15:17,3526826139003047,fraud_Johnston-Casper,travel,3.19,Nathan,Massey,M,5783 Evan Roads Apt. 465,Falmouth,...,44.2529,-85.0170,1126,Furniture designer,1955-07-06,57ff021bd3f328f8738bb535c302a31b,1371816917,44.959148,-85.884734,0


## Feature Engineering

#### Distance

In [10]:
## Here assign to n the amount of observations we have in the dataset
n = test_data.shape[0]

## Creating empty list to store the reuslts
distance_to_append = []

## Looping through each row an computing the distance between ?????????
for i in range(0,n):
    
    ## Here we gather the lat and long from the ?????
    coords_1 = (test_data['lat'][i], test_data['long'][i])
    
    ## Here we gather the lat and long from the ?????
    coords_2 = (test_data['merch_lat'][i], test_data['merch_long'][i])
    
    ## Here we compute the disance in miles between the locations
    distance_to_append.append(geopy.distance.geodesic(coords_1, coords_2).miles)

## Adding results to our data set
test_data['Distances'] = distance_to_append

In [11]:
## Visualizing the results
test_data['Distances'].head()

0    15.294273
1    65.140855
2    36.788154
3    17.159238
4    64.885553
Name: Distances, dtype: float64

#### Age

In [12]:
## Creating empty list to store the resutls
ages_to_append = []

## Looping through each observation and computing the age of each individual from its DOB
for i in range(0,n):
    
    ## Here we add the last date of this year
    year_of_2021 = datetime.strptime('2021-12-31', "%Y-%m-%d")
    
    ## Here we call each DOB in the dataset
    dob = datetime.strptime(test_data.dob[i], "%Y-%m-%d")
    
    ## Here we compute the ages
    ages_to_append.append(relativedelta(year_of_2021, dob).years)

## Adding results to our data set
test_data['Age'] = ages_to_append

In [13]:
## Visualizing the results
test_data['Age'].head()

0    53
1    31
2    51
3    34
4    66
Name: Age, dtype: int64

#### Day of the week

In [16]:
dates = pd.to_datetime(test_data['trans_date_trans_time'])
test_data['day_of_week'] = dates.dt.day_name()

In [18]:
test_data['day_of_week'].head()

0    Sunday
1    Sunday
2    Sunday
3    Sunday
4    Sunday
Name: day_of_week, dtype: object

#### Hour of the day

In [29]:
hour_of_the_day = []

for i in range(0,n):
    hour_of_the_day.append(datetime.strptime(test_data.trans_date_trans_time[i] ,"%Y-%m-%d %H:%M:%S").hour)
    
test_data['HourOfTheDay'] = hour_of_the_day

In [30]:
test_data['HourOfTheDay'].head()

0    12
1    12
2    12
3    12
4    12
Name: HourOfTheDay, dtype: int64

#### Generating unique ID

In [42]:
pd.DataFrame(pd.factorize(test_data['first'] + test_data['last'] + test_data['street'] + test_data['cc_num'].astype(str))[0]).value_counts()

19     1474
327    1466
260    1462
106    1458
10     1428
       ... 
919       9
921       9
909       8
911       7
920       6
Length: 924, dtype: int64